In [1]:
import requests
import json
import yaml
from datetime import datetime

from utils.misc import *
import re

In [2]:
headers = {
    "user-agent":
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
}

In [3]:
timestamp = datetime.now()
timestamp_str = datetime.strftime(timestamp, "%Y-%m-%d-%H")
timestamp_str

'2023-03-18-11'

In [4]:
save_paths = {'api_all_outerwear.yml': 'outerwear', 'api_coats.yml': 'coats'}


In [5]:
def format_url(url, limit=128, offset=0):
    url = re.sub(r"limit=([0-9]+)", f"limit={limit}", url)
    url = re.sub(r"offset=([0-9]+)", f"offset={offset}", url)
    return url


def get_all_items(url):
    responses = []

    queried_items = 0
    page_items = 32

    # query total data
    new_url = format_url(url, page_items, 0)
    res = requests.get(new_url, headers=headers, timeout=8)
    res = res.json()
    total = res['result']['pagination']['total']

    while queried_items < total:
        print(".", end='')

        res = requests.get(new_url, headers=headers, timeout=8)
        res = res.json()
        responses.append(res['result'])
        count = res['result']['pagination']['count']
        total = res['result']['pagination']['total']
        offset = res['result']['pagination']['offset']
        queried_items += count

        new_url = format_url(url, page_items, offset)
    print('')
    return responses

In [6]:
for config_file, save_path in save_paths.items():
    raw_data_path = f"./{save_path}/{timestamp_str}/"
    create_path_if_not_exist(raw_data_path)

    data = load_yaml(config_file)

    for country in data.keys():
        contry_data = data[country]

        country_data_path = raw_data_path + f"/{country}/"
        create_path_if_not_exist(country_data_path)

        for gender in contry_data.keys():
            url = contry_data[gender]
            try:
                print(country, gender)
                res = get_all_items(url)
                with open(country_data_path + f"{gender}.json", "w") as f:
                    f.write(json.dumps(res, indent=2, ensure_ascii=False))
            except Exception as e:
                print("Fail to request", country, gender)
                print(e)

jp women
...
jp men
...
th women
..
th men
..
au women
...
au men
..
ca women
..
ca men
..
us women
...
us men
...
jp women
.
jp men
.
th women

th men

au women
.
au men
.
ca women
.
ca men
.
us women
.
us men
.
